# Alignments' performance analysis

## Conversions to fasta

In [1]:
import pandas as pd

train = pd.read_csv('../train.csv')
#validation = pd.read_csv('../validation.csv')


KeyboardInterrupt



In [3]:
train.iloc[:, :3].to_csv('../train_reduced.csv', index=False)

In [9]:
test = pd.read_csv('../test.csv')
test.iloc[:, :3].to_csv('../test_reduced.csv', index=False)

/tmp/ipykernel_6976/1041909268.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('final_data/test.csv')


In [4]:
import pandas as pd
train = pd.read_csv('train_reduced.csv')
validation = pd.read_csv('validation_reduced.csv')
pd.concat([train, validation]).to_csv('train_validation_reduced.csv', index=False)

In [2]:
from plants_sm.utilities.utils import convert_csv_to_fasta

convert_csv_to_fasta('../train_reduced.csv', 'sequence', 'accession', 'database.fasta')

In [10]:
convert_csv_to_fasta('../test_reduced.csv', 'sequence', 'accession', 'test.fasta')

## Database creation

In [1]:
from plants_sm.alignments.alignment import Diamond

diamond = Diamond("train_database")

In [2]:
diamond.create_database('database.fasta')

diamond v2.0.14.152 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: database.fasta
Opening the database file...  [0.008s]
Loading sequences...  [0.262s]
Masking sequences...  [0.461s]
Writing sequences...  [0.05s]
Hashing sequences...  [0.021s]
Loading sequences...  [0s]
Writing trailer...  [0.002s]
Closing the input file...  [0s]
Closing the database file...  [0.212s]

Database sequences  222591
  Database letters  93658859
     Database hash  b4336dd3ec5f38ce127188226eb91b7c
        Total time  1.020000s


In [4]:
diamond.run("../test.fasta", "test_database.tsv", evalue=1e-5, num_hits=1)

diamond v2.0.14.152 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 8
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
#Target sequences to report alignments for: 1
Opening the database...  [0.05s]
Database: train_database.dmnd (type: Diamond database, sequences: 222591, letters: 93658859)
Block size = 2000000000
Opening the input file...  [0.011s]
Opening the output file...  [0s]
Loading query sequences...  [0.084s]
Masking queries...  [0.16s]
Algorithm: Double-indexed
Building query histograms...  [0.115s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.095s]
Masking reference...  [0.435s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.403s]
Allocating buffers...  [0s]
Processing query block 1, reference block 1/1, sha

In [6]:
import pandas as pd

diamond.results = pd.read_csv('test_database.tsv', sep='\t')

In [8]:
database = pd.read_csv('../train.csv')

/tmp/ipykernel_22798/2997233878.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv('../train.csv')


In [9]:
diamond.associate_to_ec(database, "test_diamond_predictions.csv")

/home/bisbii/Desktop/PHD/PlantsSM/src/plants_sm/alignments/alignment.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database_reduced.drop(columns=["sequence"], inplace=True)


## Alignments' performance analysis

In [11]:
import pandas as pd
test = pd.read_csv('../test.csv')
test.shape

/tmp/ipykernel_22798/713750523.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('../test.csv')


(75566, 2779)

In [12]:
predictions = pd.read_csv("test_diamond_predictions.csv")
predictions.shape

/tmp/ipykernel_22798/3398146152.py:1: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  predictions = pd.read_csv("test_diamond_predictions.csv")


(73877, 2788)

In [13]:
predictions.columns

Index(['qseqid', 'accession', 'pident', 'length', 'mismatch', 'gapopen',
       'qstart', 'qend', 'sstart', 'evalue',
       ...
       '7.4.2.5', '7.4.2.8', '7.5.2.11', '7.6.2.1', '7.6.2.11', '7.6.2.13',
       '7.6.2.2', '7.6.2.3', '7.6.2.5', '7.6.2.8'],
      dtype='object', length=2788)

In [14]:
predictions.drop(columns=['accession', 'pident', 'length', 'mismatch', 'gapopen',
       'qstart', 'qend', 'sstart', 'evalue', 'bitscore', 'name'], inplace=True)

In [15]:
predictions.columns

Index(['qseqid', 'EC', 'EC1', 'EC2', 'EC3', 'EC4', '1', '2', '3', '4',
       ...
       '7.4.2.5', '7.4.2.8', '7.5.2.11', '7.6.2.1', '7.6.2.11', '7.6.2.13',
       '7.6.2.2', '7.6.2.3', '7.6.2.5', '7.6.2.8'],
      dtype='object', length=2777)

In [16]:
not_in_predictions = test[~test.loc[:, "accession"].isin(predictions.loc[:, "qseqid"])]
not_in_predictions.head()

,accession,name,sequence,EC,EC1,EC2,EC3,EC4,1,2,...,7.4.2.5,7.4.2.8,7.5.2.11,7.6.2.1,7.6.2.11,7.6.2.13,7.6.2.2,7.6.2.3,7.6.2.5,7.6.2.8
129,P83855,ACLP_MOUSE,GVYMEIGRCRXEAXRRRKEAV,4.6.1.1,4,4.6,4.6.1,4.6.1.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,O53582,AFTB_MYCTU,MVRVSLWLSVTAVAVLFGWGSWQRRWIADDGLIVLRTVRNLLAGNG...,2.4.2.-,2,2.4,2.4.2,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,P82955,AHPF_ACICA,MLDQNIXTQLXAYLER,1.8.1.-,1,1.8,1.8.1,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,B7LM79,AIS_ESCF3,MKLKKHVAVLLISFLCLIGLVTQHAWSSNGLPRIDNKTLAKLSLQH...,3.1.3.-,3,3.1,3.1.3,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
255,Q8Z543,AIS_SALTI,MLAFTLRFIKNKRYFAILAGALVIIAGLASQHAWSGNGLPQINGKA...,3.1.3.-,3,3.1,3.1.3,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
not_in_predictions.iloc[:, 8:] = 0.0

/tmp/ipykernel_22798/447101761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_predictions.iloc[:, 8:] = 0.0


In [18]:
not_in_predictions.drop(["sequence", "name"], axis=1, inplace=True)

/tmp/ipykernel_22798/4052792183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_predictions.drop(["sequence", "name"], axis=1, inplace=True)


In [19]:
not_in_predictions.columns = predictions.columns

In [20]:
predictions = pd.concat([predictions, not_in_predictions])

In [21]:
predictions.shape

(75566, 2777)

In [22]:
predictions.columns

Index(['qseqid', 'EC', 'EC1', 'EC2', 'EC3', 'EC4', '1', '2', '3', '4',
       ...
       '7.4.2.5', '7.4.2.8', '7.5.2.11', '7.6.2.1', '7.6.2.11', '7.6.2.13',
       '7.6.2.2', '7.6.2.3', '7.6.2.5', '7.6.2.8'],
      dtype='object', length=2777)

In [23]:
predictions.sort_values(by=['qseqid'], inplace=True)

In [24]:
test.sort_values(by=['accession'], inplace=True)

In [25]:
predictions.loc[:, "qseqid"].tolist() == test.loc[:, "accession"].tolist()

True

In [26]:
predictions.to_csv("test_diamond_predictions_right_format.csv", index=False)
test.to_csv("test_right_format.csv", index=False)

In [1]:
import pandas as pd

predictions = pd.read_csv("test_diamond_predictions_right_format.csv")
test = pd.read_csv("test_right_format.csv")

/tmp/ipykernel_23479/342144657.py:3: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  predictions = pd.read_csv("test_diamond_predictions_right_format.csv")
/tmp/ipykernel_23479/342144657.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("test_right_format.csv")


In [2]:
predictions.columns

Index(['qseqid', 'EC', 'EC1', 'EC2', 'EC3', 'EC4', '1', '2', '3', '4',
       ...
       '7.4.2.5', '7.4.2.8', '7.5.2.11', '7.6.2.1', '7.6.2.11', '7.6.2.13',
       '7.6.2.2', '7.6.2.3', '7.6.2.5', '7.6.2.8'],
      dtype='object', length=2777)

In [4]:
swiss_prot = pd.read_csv("../swiss_prot_ec_filtered.csv")

In [5]:
predictions_swiss_prot = predictions[predictions.loc[:, "qseqid"].isin(swiss_prot.loc[:, "accession"])]

In [6]:
test_swiss_prot = test[test.loc[:, "accession"].isin(swiss_prot.loc[:, "accession"])]

In [7]:
predictions_swiss_prot.loc[:, "qseqid"]

0        A0A009IHW8
19       A0A011QK89
37       A0A017SEY2
38       A0A017SP50
63       A0A023I7E1
            ...    
75505        W8JWV8
75506        W8JWW7
75532        X1WBB5
75533        X2EVV9
75535        X2JE85
Name: qseqid, Length: 53167, dtype: object

In [8]:
predictions.iloc[:, 6:].head()

,1,2,3,4,5,6,7,1.1,1.10,1.11,...,7.4.2.5,7.4.2.8,7.5.2.11,7.6.2.1,7.6.2.11,7.6.2.13,7.6.2.2,7.6.2.3,7.6.2.5,7.6.2.8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
zero_columns = test_swiss_prot.iloc[:, 8:].columns[(test_swiss_prot.iloc[:, 8:] == 0).all()]
columns = list(test_swiss_prot.iloc[:, 8:].columns)
for column in zero_columns: 
       columns.remove(column)
test_swiss_prot_reduced = test_swiss_prot.iloc[:, 8:].loc[:, columns]
predictions_swiss_prot_reduced = predictions_swiss_prot.iloc[:, 6:].loc[:, columns]

In [10]:
predictions_swiss_prot_reduced.head()

,1,2,3,4,5,6,7,1.1,1.10,1.11,...,7.4.2.5,7.4.2.8,7.5.2.11,7.6.2.1,7.6.2.11,7.6.2.13,7.6.2.2,7.6.2.3,7.6.2.5,7.6.2.8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Metrics

In [11]:
from sklearn.metrics import f1_score

f1_score(test_swiss_prot_reduced, predictions_swiss_prot_reduced, average='macro')

0.8676778409804393

In [12]:
from sklearn.metrics import precision_score

precision_score(test_swiss_prot_reduced, predictions_swiss_prot_reduced, average='macro', zero_division=1)

0.877754147536592

In [13]:
from sklearn.metrics import recall_score

recall_score(test_swiss_prot_reduced, predictions_swiss_prot_reduced, average='macro')

0.9058346359325334

In [18]:
from sklearn.metrics import f1_score

f1_score(test.iloc[:, 8:], predictions.iloc[:, 6:], average='macro', zero_division=1)

0.8894964697370696

In [15]:
from sklearn.metrics import precision_score

precision_score(test.iloc[:, 8:], predictions.iloc[:, 6:], average='macro', zero_division=1)

0.9022298961300323

In [16]:
from sklearn.metrics import recall_score

recall_score(test.iloc[:, 8:], predictions.iloc[:, 6:], average='macro', zero_division=1)

0.8858586826680179